# Milvus测试

In [1]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_demo.db")


I0000 00:00:1735616245.766444 1872685 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1735616245.772386 1872808 subchannel.cc:806] subchannel 0x12b9afbf0 {address=unix:/var/folders/rn/5219x28n1bb9p3w3td_571000000gn/T/tmpr6mzfg3t_milvus_demo.db.sock, args={grpc.client_channel_factory=0x15b671680, grpc.default_authority=var%2Ffolders%2Frn%2F5219x28n1bb9p3w3td_571000000gn%2FT%2Ftmpr6mzfg3t_milvus_demo.db.sock, grpc.enable_retries=1, grpc.internal.channel_credentials=0x15b66dc00, grpc.internal.client_channel_call_destination=0x10eebba18, grpc.internal.event_engine=0x13bd4b3f0, grpc.internal.security_connector=0x12b987c40, grpc.internal.subchannel_pool=0x15b6a5400, grpc.keepalive_time_ms=55000, grpc.max_receive_message_length=-1, grpc.max_send_message_length=-1, grpc.primary_user_agent=grpc-python/

In [2]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)


In [7]:
from pymilvus import model
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

model_path = '../models/gte-base-en-v1.5'
model = SentenceTransformer(model_path, trust_remote_code=True)

docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = model.encode(docs)
print("Dim:", vectors[0].shape)  # Dim: 768 (768,) 

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


Dim: (768,)
Data has 3 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 768


In [9]:
res = client.insert(collection_name="demo_collection", data=data)

print(res)


{'insert_count': 3, 'ids': [0, 1, 2]}


In [10]:
query_vectors = model.encode(["Who is Alan Turing?"])

res = client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)


data: ["[{'id': 1, 'distance': 0.7142452001571655, 'entity': {'text': 'Alan Turing was the first person to conduct substantial research in AI.', 'subject': 'history'}}, {'id': 2, 'distance': 0.643290102481842, 'entity': {'text': 'Born in Maida Vale, London, Turing was raised in southern England.', 'subject': 'history'}}]"] 
